In [ ]:
!pip install langchain openai faiss-cpu flask requests beautifulsoup4

In [ ]:
!pip install langchainhub transformers torch sentence-transformers llama-cpp-python

In [ ]:
!pip install -U langchain_community

In [ ]:
!pip install flask_ngrok pyngrok

In [ ]:
import os
import requests
from flask import Flask, request, jsonify
from pyngrok import ngrok
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.chains.question_answering import load_qa_chain
import threading


In [ ]:
url = "https://brainlox.com/courses/category/technical"
loader = WebBaseLoader(url)
documents = loader.load()
print(f"Extracted {len(documents)} documents from {url}")

Extracted 1 documents from https://brainlox.com/courses/category/technical


In [ ]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(documents, embedding)
vector_store_path = "/content/vector_store"
vector_db.save_local(vector_store_path)

<ipython-input-9-d57eaa581244>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

In [ ]:
#hf_pBondMcL#################
from huggingface_hub import login
login()

In [ ]:
!wget --header="Authorization: Bearer hf_p###############" \
     -O /content/mistral.gguf "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf"


--2025-02-08 11:31:30--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1739017891&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTAxNzg5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2Y

In [ ]:
!ngrok authtoken 2skmM4EjJw9######################

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
app = Flask(__name__)
vector_db = FAISS.load_local(vector_store_path, embedding,allow_dangerous_deserialization=True)
llm = LlamaCpp(model_path="/content/mistral.gguf", temperature=0.7)
retriever = vector_db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    chain_type="stuff"  # Ensures proper document combination
)
@app.route('/chat', methods=['POST'])
def chat():
    question = request.json.get('question')
    answer = qa_chain.invoke({"question":question})
    return jsonify({'answer': answer})
def run_flask():
  app.run(port=5000)
thread = threading.Thread(target=run_flask)
thread.start()
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/mistral.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_coun

 * Serving Flask app '__main__'
 * Debug mode: off


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'mistralai_mistral-7b-instruct-v0.1', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '1', 'llama.attention.head_count': '32', 'llama.block_count': '32', 'llama.attention.head_count_kv': '8', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: llama-2
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Public URL: https://8112-34-16-163-96.ngrok-free.app
https://8a2bm37hdlr-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
import requests

# Replace with your ngrok URL from the previous step
api_url = "https://8112-34-16-163-96.ngrok-free.app/chat"

# Send a question to the chatbot
response = requests.post(api_url, json={"question": "What is AI?"})
if response.status_code == 200:
  print(response.text)
  # Print the chatbot's response
  print(response.json())
else:
  print(f"Error: Request failed with status code {response.status_code}")
  print(response.text)

ERROR:__main__:Exception on /chat [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-13-5a8b7497b8a5>", line 17, in chat
    answer = qa_chain.invoke({"query":question})
             ^^^^^^^^^^^

Error: Request failed with status code 500
<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

